<h1>Numpy and Pandas Home Assignment</h1>
In this assignment, we will use pandas and numpy to do some fairly basic analysis on equities.

<h3>Imports</h3>

In [1]:
import datetime
import numpy as np
import pandas as pd
%matplotlib inline
import pandas_datareader.data as web
import fix_yahoo_finance as yf

<h3>Get data on the following four stocks from yahoo finance</h3>
<li>MS, GS, JPM, C
<li>Use the date range 2013/1/1/ to 2018/9/24

In [2]:
stock_data = web.DataReader(['MS','GS','JPM','C'],'yahoo',datetime.datetime(2013,1,1),datetime.datetime(2018,9,24))['Close']
stock_data

Symbols,MS,GS,JPM,C
Date,,,,
2012-12-31,19.120001,127.559998,43.970001,39.560001
2013-01-02,19.620001,131.660004,44.660000,41.250000
2013-01-03,19.580000,130.940002,44.570000,41.389999
2013-01-04,20.190001,134.509995,45.360001,42.430000
2013-01-07,19.799999,134.259995,45.410000,42.470001
...,...,...,...,...
2018-09-18,47.849998,228.889999,114.300003,71.360001
2018-09-19,49.099998,235.580002,117.620003,73.720001
2018-09-20,49.880001,237.399994,118.629997,74.790001


<h3>Compute the cross correlation coefficients on the stocks</h3>
<li>Create a df 'rets' with the one day percentage changes
<li>Calculate cross correlations on the rets df

In [3]:
rets = stock_data.pct_change()
rets.corr()

Symbols,MS,GS,JPM,C
Symbols,,,,
MS,1.000000,0.844553,0.801439,0.810056
GS,0.844553,1.000000,0.813269,0.792937
JPM,0.801439,0.813269,1.000000,0.849128
C,0.810056,0.792937,0.849128,1.000000


<h3>Select a pair of stocks to trade</h3>
<li>Choose the pair with the highest correlation
<li>You can just eyeball the result and pick the highest correlation pairs
<li>Create a new df 'pairs' with the returns columns of the highest correlation pairs

In [4]:
pairs = rets[['JPM','C']]
pairs

Symbols,JPM,C
Date,,
2012-12-31,NaN,NaN
2013-01-02,0.015692,0.042720
2013-01-03,-0.002015,0.003394
2013-01-04,0.017725,0.025127
2013-01-07,0.001102,0.000943
...,...,...
2018-09-18,0.004041,0.006772
2018-09-19,0.029046,0.033072
2018-09-20,0.008587,0.014514


<h3>Calculate the mean and std dev of the columns (returns of the two stocks)</h3>
<li>Note that line 1 contains a NaN. You'll have to get rid of this line
<li>df.describe() returns summary stats for all cols in a dataframe. You can use this to extract means and standard deviations

In [5]:
new_df = rets[['C','JPM']].iloc[1:]
c_std = new_df.describe()['C']['std']
j_std = new_df.describe()['JPM']['std']
c_mean = new_df.describe()['C']['mean']
j_mean = new_df.describe()['JPM']['mean']

<h3>Create a new column "Long_JPM_Short_C"</h3>
<li>If the return of C is greater than its mean + 0.25* std
<li> AND
<li>If the return of JPM is less than its mean - 0.25*std
<li>The value in this col should be True or False

In [6]:
new_df['Long_JPM_Short_C'] = (new_df['C'] > (c_mean + 0.25 * c_std)) & (new_df['JPM'] < (j_mean - 0.25 * j_std))
new_df

Symbols,C,JPM,Long_JPM_Short_C
Date,,,
2013-01-02,0.042720,0.015692,False
2013-01-03,0.003394,-0.002015,False
2013-01-04,0.025127,0.017725,False
2013-01-07,0.000943,0.001102,False
2013-01-08,-0.000236,0.001982,False
...,...,...,...
2018-09-18,0.006772,0.004041,False
2018-09-19,0.033072,0.029046,False
2018-09-20,0.014514,0.008587,False


<h3>Create a new column "Long_C_Short_JPM"</h3>
<li>If the return of JPM is greater than its mean + 0.25* std
<li> AND
<li>If the return of C is less than its mean - 0.25*std
<li>The value in this col should be True or False

In [7]:
new_df['Long_C_Short_JPM'] = (new_df['JPM'] > (j_mean + 0.25 * j_std)) & (new_df['C'] < (c_mean - 0.25 * c_std))
new_df

Symbols,C,JPM,Long_JPM_Short_C,Long_C_Short_JPM
Date,,,,
2013-01-02,0.042720,0.015692,False,False
2013-01-03,0.003394,-0.002015,False,False
2013-01-04,0.025127,0.017725,False,False
2013-01-07,0.000943,0.001102,False,False
2013-01-08,-0.000236,0.001982,False,False
...,...,...,...,...
2018-09-18,0.006772,0.004041,False,False
2018-09-19,0.033072,0.029046,False,False
2018-09-20,0.014514,0.008587,False,False


<h3>Create a new column "Trade_return"</h3>
<li>If Long_JPM_Short_C is True and Long_C_Short_JPM is False, the value of this column should be the return on JPM minus the return on C
<li>If Long_JPM_Short_C is False and Long_C_Short_JPM is True, the value of this column should be the return on C minus the return on JPM
<li>Otherwise the value should be zero

In [8]:
new_df['Trade_return'] = np.where((new_df['Long_JPM_Short_C']&~(new_df['Long_C_Short_JPM'])),new_df['JPM']-new_df['C'],np.where((new_df['Long_C_Short_JPM']&~(new_df['Long_JPM_Short_C'])),new_df['C']-new_df['JPM'],0))
new_df

Symbols,C,JPM,Long_JPM_Short_C,Long_C_Short_JPM,Trade_return
Date,,,,,
2013-01-02,0.042720,0.015692,False,False,0.0
2013-01-03,0.003394,-0.002015,False,False,0.0
2013-01-04,0.025127,0.017725,False,False,0.0
2013-01-07,0.000943,0.001102,False,False,0.0
2013-01-08,-0.000236,0.001982,False,False,0.0
...,...,...,...,...,...
2018-09-18,0.006772,0.004041,False,False,0.0
2018-09-19,0.033072,0.029046,False,False,0.0
2018-09-20,0.014514,0.008587,False,False,0.0


<h3>Calculate the sum of this new column</h3>
<li>Are you going to get rich?

In [9]:
new_df['Trade_return'].sum()

-0.6793516982785628

In [10]:
print("NO!")

NO!
